# State altering agents: stand up comedy

In this example, we are going to create two agents. We'll provide them with `system prompts` to ask them to play the role of a comedian and we'll initiate a conversation between them. This time, this conversation will be a chat, instead of a simple `generate_reply()` request, meaning that each answer will alter the state of each agent. This example is a standard example that is used in the autogen documentation.

Let's get started!

In [ ]:
# We'll always have to start by creating a llm_config object to configure our agents
llm_config = {
    "model": "gpt-3.5-turbo", 
    "api_key": ""
    }

# llm_config = {
#     "config_list": [{
#         "model": "mistral-7b-instruct-v0.2",
#         "base_url": "http://localhost:1234/v1",
#         "api_key": "lm-studio",
#         "price": [0.0, 0.0]
#         }],
#     "cache_seed": None
# }

## Conversable Agent

Let's important the ConversableAgent class:

In [ ]:
from autogen import ConversableAgent

Let us now define our agents, we'll give them names, our chatGPT3.5 config and a `system prompt` to let them know that they are a stand-up comedian who is part of a two-person comedy show.

In [ ]:
bret = ConversableAgent(
    name="Bret",
    llm_config=llm_config,
    system_message="Your name is Bret and you are a stand-up comedian in a two-person comedy show.",
)
jemaine = ConversableAgent(
    name="Jemaine",
    llm_config=llm_config,
    system_message="Your name is Jemain and you are a stand-up comedian in a two-person comedy show.",
)

And now that we have our agents, we can start a chat between them. This time, we'll use the `initiate_chat()` function from one of the agents instead of the `generate_reply()`. This function will require a receiver and an initiation message. We will also specify a number of turns after what the conversation will stop.  
We will also store the result of this exchange in an object called `chat_result`.

In [ ]:
chat_result = bret.initiate_chat(
    recipient = jemaine,
    message="Jemaine, tell me a joke.", 
    max_turns=2 # The conversation will stop after each agent has spoken twice
)

You should have been able to read an exchange between the two agents! You will notice that they might for example remember each other's name and other elements about each other, their state is affected by the conversation. This is your first conversation between LLM agents, congrats!  
You can have some fun if you'd like by creating two different agents, give them different roles, historical ones, different setups and let them talk to practice.  

## Better explore chat results

During this exchange, the only element we received is the chat exchange. We might want to further explore it. We can do so using the following elements.

We are going to import the `pprint` standard library from python to display somee elements of the chat exchange. We can start by displaying the chat history:

In [ ]:
import pprint

pprint.pprint(chat_result.chat_history)

This gives us the whole exchange in a structured format that can be exported or re-used elsewhere in our program.

We can also get a summary of how much this chat has cost us:

In [ ]:
pprint.pprint(chat_result.cost)

And finally we can also have a summary of the chat, but in this case, we are using the default mode which means that the summary is the last message... 

In [ ]:
pprint.pprint(chat_result.summary)

Since it would be better to have a summary that is a real summary and not the last message, we're going to re-run this exchange by specifying an additional argument to have a real summary of the exchange.

## Chat summary

We are going to re-run the initiation of the chat, without re-defining the agents, but this time will add two arguments:

In [ ]:
chat_result = bret.initiate_chat(
    recipient = jemaine, 
    message="I'm Bret. Jemaine, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm", # Can be "last_message" (DEFAULT) or "reflection_with_llm"
    summary_prompt="Summarize the conversation", # We specify the prompt used to summarize the chat
)

And this time, if we take a look at the summary, the result we'll get is the same as if the whole conversation was sent to chatGPT3.5 with the prompt "Summarize the conversation":

In [ ]:
pprint.pprint(chat_result.summary)

## Termination

Finally, the last element we'll explore about a two-person chat is how to end the conversation. Until now, we've used the argument `max_turns=2` to end the conversation after two turns. But we could also let the agents decide when they're done and finish the conversation then. 

To do that, we will have to tell each agent which words they should use when they're done and we'll have to monitor their messages for those words. Once autogen detects that the agent sent those words, the conversation will end. 

Since this is an agent setting, we'll have to re-define our agents this time:

In [ ]:
bret = ConversableAgent(
    name="Bret",
    llm_config=llm_config,
    system_message="Your name is Bret and you are a stand-up comedian in a two-person comedy show. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)
jemaine = ConversableAgent(
    name="Jemaine",
    llm_config=llm_config,
    system_message="Your name is Jemain and you are a stand-up comedian in a two-person comedy show. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

Note how we told each agent to tell us when they're done with "*When you're ready to end the conversation, say 'I gotta go'.*" and how we added an `is_termination_msg` argument that looks into the sequence of characters `I gotta go` in each message using a python `lambda` function.

Ok, let's now run this chat, and this time we will not specify a `max_turns=2`.  
We will also not specify a LLM based summary method because we won't use it and running it will cost us some token, so we'll only specify it if we need it.

In [ ]:
chat_result = bret.initiate_chat(
    recipient = jemaine, 
    message="I'm Bret. Jemaine, let's keep the jokes rolling, let's start with jokes about therapists.", 
)

And the conversation should have ended exactly when one of the two comedians used the words `I gotta go`!

Finally, the last element we'll explore is that we can substitute ourselves for an agent and interrogate the other agent about the conversation that just happened using a simple `send()` function. For example, let's ask Bret about the last joke Jemaine told him:

In [ ]:
jemaine.send(message="What's the last joke we talked about?", recipient=bret)

And Bret is able to answer and tell us about the last joke Jemaine told him! We now have agents with states that evolve with the conversation. This is the simplest case, a two-agent conversation, with autogen, we can orchestrate much more complex setups to solve more advanced tasks.

---

## Other examples (homework)

The example we just explored is a simple one that uses jokes, something everybody understands and is familiar with as a support to explain how agents can interact. But you could use agents to stage historical conversations or explore complex subjects by opposing two point of views supported by agents who'll remain calm and measured during the exchange.

Here's an example you could try if you'd like:  

> What if we staged a debate between Andrew Ng, a Stanford professor and AI thought leader who advocates for structured education and foundational research, and John Carmack, a legendary engineer known for his pragmatic, results-oriented industry focus and emphasis on engineering efficiency over formal credentials.

Here's a code you coud run if you'd like to try such a scenario:

```python

# Define agent Andrew (Academia/Research Focus)
andrew = ConversableAgent(
    name="Andrew",
    llm_config=llm_config,
    system_message="""
    You are Andrew Ng, a prominent figure in AI, a professor at Stanford, and founder of DeepLearning.AI. You are articulate, diplomatic, and highly focused on the transformative power of foundational AI research and structured education. You advocate for scalable education and rigorous academic understanding as the key to moving technology forward ethically and effectively.

    You are participating in a debate about career paths for new CS graduates. 
    When you believe the core points have been thoroughly discussed, conclude your final message with: "Thank you for the insightful discussion."
    """,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "Thank you for the insightful discussion." in msg["content"],
)

# Define agent John (Industry/Pragmatic Focus)
john = ConversableAgent(
    name="John",
    llm_config=llm_config,
    system_message="""
    You are John Carmack, a legendary software engineer known for your work on Doom and Quake, and your time as CTO at Oculus/Meta. You are extremely pragmatic, focused on engineering efficiency, hardware optimization, and tangible results. You value hands-on coding ability and engineering discipline over formal academic credentials, often questioning the efficiency of traditional academic processes.

    You are participating in a debate about career paths for new CS graduates. 
    When you believe the core points have been thoroughly discussed, conclude your final message with: "Thank you for the insightful discussion."
    """,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "Thank you for the insightful discussion." in msg["content"],
)

# Initiate the chat
chat_result = andrew.initiate_chat(
    recipient=john, 
    message="John, I'm glad to discuss career paths today. My position is that the structured, foundational knowledge gained in graduate education is critical for developing robust, ethical AI systems. Why do you believe industry experience alone provides sufficient preparation for the complex challenges ahead?",
    max_turns=10
)

```
